# Transaction Cost analysis

for different kernels

# Libraries and Data

In [1]:
import numpy as np
import pandas as pd
import sys
import os
import pickle
import cvxpy as cp
from scipy.optimize import minimize
from project_lib.backtest import *
from project_lib.utils import *
from project_lib.performance import *
from project_lib.analysis import *
from project_lib.portfolio import Portfolio
from project_lib.backtest import *

HOME_DIRECTORY = 'C:/Users/Harol/OneDrive/Documents/master computational finance/thesis/thesis_UCL/Code/Transaction Costs'
sys.path.append(HOME_DIRECTORY)

In [2]:
# import returns
with open(HOME_DIRECTORY + '/data/processed_daily_data/ret_subset.pkl', 'rb') as f:
    ret = pickle.load(f)

In [3]:
universe_size = 100
ret = ret.iloc[:, :universe_size]  # subset the data
ret = ret.iloc[(4):] # burn


In [4]:
prices = (1 + ret).cumprod()
prices = prices.iloc[:,:universe_size]

In [ ]:
# import weights and correlations
gauss_w = pd.read_csv("gaussian_weights.csv")
tri_w = pd.read_csv("triangular_weights.csv")
epan_w = pd.read_csv("epanechnikov_weights.csv")
cca_w = pd.read_csv("sample_cca_weights.csv") # linear version

gauss_c = pd.read_csv("gaussian_correlations.csv")
tri_c = pd.read_csv("triangular_correlations.csv")
epan_c = pd.read_csv("epanechnikov_correlations.csv")

dfs = [gauss_w, tri_w, epan_w, gauss_c, tri_c, epan_c]

In [ ]:
for df in dfs:
    df.set_index("date", inplace=True)
    df.index = pd.to_datetime(df.index)

In [ ]:
cca_w.set_index("Unnamed: 0", inplace=True)
cca_w.index = pd.to_datetime(cca_w.index)

In [ ]:
kernels = ["gaussian","triangular","epanechnikov"]

## preprocessing

In [ ]:
def cleaning(x):
    a = [ele for ele in x.strip("[]").split(" ") if ele.strip()]
    a = [elem.replace("\n","") for elem in a]
    return np.asarray(a,dtype=float)

In [ ]:
gauss_w  = gauss_w.applymap(cleaning)
tri_w = tri_w.applymap(cleaning)
epan_w = epan_w.applymap(cleaning)

## correcting a mistake regarding the correlations

In [ ]:
K_pl = np.linalg.inv(cov_R_half) @ (results.T @ results) @ np.linalg.inv(cov_R_half)
        
        # make sure it's sorted
        eigen_val, eigen_vec = np.linalg.eig(K_pl)
        order = np.argsort(eigen_val)[::-1]
        idx = np.empty_like(order)
        idx[order] = np.arange(len(order))
        eigen_vec[:] = eigen_vec[:, idx] 

In [ ]:
# step 1: all column elements as one matrix
# step 2: eigenvalues
# step 3: eigenvalues in place
#for kernel in kernels:
for kernel in kernels:
    print(kernel)
    cp = all_weights[kernel].copy()
    for row in range(cp.shape[0]):
        cov_R_half = get_cov(ret.iloc[row:(row+250),:], method="sample", square_root=True)
        # re-extract the weights
        w = cov_R_half.T @ np.array(gauss_w.iloc[0,:].to_list()).T
        
        K = w @ w.T
        
        eigenval, eigenvec = np.linalg.eig(K)
        
        for col in range(all_corr[kernel].shape[1]):
            all_corr[kernel].iloc[row,col] = eigenval[col]


# Transaction cost based on portfolios

Inspired from analytical solutions of optimal portfolio rebalancing, Ding Liu, 2019

In [ ]:
all_weights = {'gaussian':gauss_w, 'triangular':tri_w,'epanechnikov':epan_w}
all_corr = {'gaussian':gauss_c, 'triangular':tri_c,'epanechnikov':epan_c}

In [ ]:
def update_weights(new_weights,old_weights, corr, tcost, risk_aversion, pf_variance):
    """
        rebalance the weights depending on rebalancing costs, risk aversion, and portfolio variance
        
        inputs:
                new_weights   : matrix of all weights calculated using CCA at t [m x m]
                old_weights   : total portfolio weights at t-1                  [1 x m]
                corr          : correlations corresponding to new_weights at t  [1 x m]
                tcost         : transaction cost parameter                      [1 x 1]
                risk_aversion : risk aversion parameter                         [1 x 1]
                pf_variance   : current portfolio variance at t-1               [1 x 1]
        outputs:
                change_w : change in weights (compared to old weights) [1 x m]
    """
    # step 1 : multiply each portfolio by its correlation
    w = new_weights*np.array(corr)
    # step 2 : calculate total weights
    total_w = w.sum()
    # step 3 : calculate weights change
    change_w = old_weights - total_w
    # step 4 : initial transaction cost
    rebalance_cost = tcost * np.sum(np.abs(change_w))
    # step 5 : calculate which portfolios to include iteratively
    trade = False
    while(trade==False):
        # check whether it is worth rebalancing given total current turnover
        if np.sum(np.abs(change_w)) - (1/risk_aversion) * rebalance_cost / pf_variance < 0:
            # remove last canonical portfolio
            # unless we decide not to trade any portfolio -> end loop
            if w.shape[1]==0:
                trade = True
            else:
                w = np.delete(w, -1, 1)
                total_w = w.sum()
                change_w = old_weights - total_w
                rebalance_cost = tcost * np.sum(np.abs(change_w))
        else:
            trade = True
    # return the change in weights
    return change_w

In [ ]:
# function to iteratively calculate portfolio weights & profitability
def backtest_cca():
    return 0

# Transaction cost on asset level

Implementation of "Multiperiod portfolio optimization with multiple risky assets and general transaction costs", Mei, Demiguel, Nogales, 2016

In [5]:
def rebalancing(X,X_prev, rho, gamma, kappa, mu,sigma, lag, target="Markowitz"):
    """
        Function to calculate optimal rebalancing on asset level with proportional transaction costs.
        
        Implementation equation (2) in Multiperiod portfolio optimization with multiple risky assets
        and general transaction costs.
        
        Inputs:
                X      : target weights                   [1 x m]
                X_prev : previous weights                 [1 x m]
                rho    : discount rate                    [1 x 1]
                gamma  : absolute risk-aversion parameter [1 x 1]
                kappa  : transaction cost parameter       [1 x 1]
                mu     : mean returns                     [1 x m]
                sigma  : covariance of returns            [m x m]
                lag    : rebalancing horizon              [1 x 1]
        Output:
                new_w : new weights [1 x m]
    """
    
    if target=="Markowitz":
        constraints = []
        m = len(X)
        # initiliase variable
        w = cp.Variable(m)
        # objective function
        obj = cp.Maximize((1-rho)**lag * (w * mu - gamma/2 * w * sigma * w) - kappa*cp.norm(w - X_prev, 1))
        prob = cp.Problem(obj, constraints)
        prob.solve(verbose = False)
        new_w = np.array(w.value)
        
    elif target=="Target":
        # Calculated using SCIPY (CVXPY does not support formulation)
        arguments = (X, kappa, X_prev)
        res = minimize(minimize_target, x0=X, args=arguments)
        new_w = res.x
        
    elif target == "Tradeoff":
        # Calculated using SCIPY (CVXPY does not support formulation)
        arguments = (X, X_prev, gamma, kappa, sigma)
        res = minimize(minimize_tradeoff, x0 = X, args = arguments)
        new_w = res.x

    return new_w

def minimize_target(w, w_target, tcost, w_prev):
    """
        minimizes difference between target weights and actual weights whilst penalizing for difference with previous weights
        
        Equation : w_target - w + tcost * |w - w_prev|
        
        inputs:
                w        : actual weights                   [1 x m]
                w_target : target weights                   [1 x m]
                w_prev   : previous weights                 [1 x m]
                tcost  : transaction cost parameter         [1 x 1]
        outputs:
                norm1 of Equation
    """
    return np.linalg.norm(w_target-w + tcost * np.abs(w - w_prev),1)

def minimize_tradeoff(w,w_target,w_prev,gamma,tcost,covar):
    """
        minimizes difference between target weights and actual weights whilst penalizing for
        the difference with previous weights. Taking into account the covariance matrix, risk aversion (tracking error)
        and transaction cost parameter.
        
        Equation 1 in "Analytical solutions of optimal portfolio rebalancing", Ding Liu, 2019
        
        inputs:
                w        : actual weights                   [1 x m]
                w_target : target weights                   [1 x m]
                w_prev   : previous weights                 [1 x m]
                gamma  : absolute risk-aversion parameter   [1 x 1]
                tcost  : transaction cost parameter         [1 x 1]
                covar  : covariance of returns              [m x m]
        outputs:
                norm1 of Equation 1
        
    """
    # norm1[ 1/(2*gamma) * (w - w_T) @ covar @ (w-w_T)' + tcost * (w - w_(t-1))' ]
    return np.linalg.norm( (1 / (2*gamma)) * (w - w_target) @ covar @ (w - w_target).T + tcost * np.abs(w - w_prev), 1)

In [6]:
def constant_rebalancing(weights, rho, gamma, kappa, returns, lag, target):
    """
        function to perform continuous rebalancing taking into account transaction costs
    
    """
    # create some variables    
    means = returns.rolling(250).mean().iloc[250:,:]
    #covariances = returns.rolling(250).cov()
    covariances = 1
    new_weights = weights.copy()
    
    new_weights.iloc[0,:] = new_weights.iloc[0,:]
    
    # first very basic function
    for i in range(1,weights.shape[0]):
        if i % 50==0:print("iteration {}".format(i))
        target_w = np.array(weights.iloc[i,:])
        prev_w = np.array(new_weights.iloc[i-1,:])
        
        covariances = get_cov(np.array(returns.iloc[i:(i+250),:]), method="nls", square_root=False)

        temp = rebalancing(target_w, prev_w, rho=rho,
                              gamma=gamma, kappa=kappa, mu=means.iloc[i,:], sigma=covariances, lag=lag, target=target)
        
        for j in range(len(temp)):
            new_weights.iloc[i,j] = temp[j]
    
    return new_weights


## impact of different levels of transaction costs

In [7]:
tcosts = [0.0001,0.0002,0.0003,0.0005,0.001, 0.0025, 0.005, 0.01]


In [ ]:
def rebalancing_output_tcosts(naming,tcosts, cca_w, rho, gamma, returns, lag=1, target="Tradeoff"):
    for tcost in tcosts:
        print("on tcost {}".format(tcost))
        tcost_weights = constant_rebalancing(cca_w, rho=rho, gamma=gamma,
                                             kappa=tcost, returns=returns, lag=lag, target=target)
        tcost_weights.to_csv(naming+"_"+str(tcost)+".csv")

### import previously extracted files

In [50]:
def extract_tcost_files(tcost_list, base_name,sample_name,include_sample = True):
    
    # extract data frames and put them into a list
    list_of_files = [pd.read_csv(base_name+str(i)+".csv") for i in tcost_list]
    if include_sample:
        assert len(sample_name) != 0, "if you want to include sample cca name, include its file names" 
        list_of_files.insert(0,pd.read_csv(sample_name))
        
    # some quick preprocessing
    for df in list_of_files:
        if "Unnamed: 0" in df.columns:
            df.set_index("Unnamed: 0", inplace=True)
            df.index = pd.to_datetime(df.index)
            df.index.names = ['date']
        elif "date" in df.columns:
            df.set_index("date", inplace=True)
            df.index = pd.to_datetime(df.index)
            df.index.names = ['date']
        
    return list_of_files

def list_to_dict(keys, lst):
    return dict(zip(transaction_costs,cca_tcosts))

In [51]:
# assign dataset names
tcosts = [0.0005, 0.001, 0.0025, 0.005, 0.01, 0.05, 0.1]
base_name = "cca_tcostweights_nls_tradeoff_"
cca_tcosts = extract_tcost_files(tcosts,base_name,"sample_cca_weights.csv", include_sample=True)

# create dictionary from list
transaction_costs = [str(i) for i in tcosts]
transaction_costs.insert(0,"sample")
cca_dict = list_to_dict(transaction_costs, cca_tcosts)

### evaluate performance

dictionary to use is *cca_dict*

In [56]:
pnl_results = dict()
ptf_ret = dict()
for tc in cca_dict:
    # we turn 'tc' into a float, which is why there is a separation
    if tc!="sample":     
        portfolio =  Portfolio(prices=prices.loc[cca_dict[tc].index], position=cca_dict[tc], period=0,tcost=np.float(tc))
        ptf_ret[tc] = portfolio.adjusted_profit.to_frame(name="Profit")
        pnl_results[tc] = portfolio.adjusted_nav().to_frame(name="NAV")
    else:
        portfolio =  Portfolio(prices=prices.loc[cca_dict[tc].index], position=cca_dict[tc], period=0,tcost=0)
        ptf_ret[tc] = portfolio.adjusted_profit.to_frame(name="Profit")
        pnl_results[tc] = portfolio.adjusted_nav().to_frame(name="NAV")

In [57]:
build_table2(transaction_costs, ptf_ret)

,AV,SD,IR,VaR,MDD,P2T,P2P,P2PL,Calmar,Stability,Omega,Sortino,TailRatio,CSR,Kurtosis
sample,182.19,47.67,3.82,0.22,-1.00,1.0,70.0,70.0,1.82,0.97,1.92,6.88,1.55,2.98,2.08
0.0005,60.92,47.16,1.29,0.22,-1.00,1.0,224.0,224.0,0.61,0.86,1.25,2.02,1.22,1.52,2.16
0.001,-30.87,46.67,-0.66,0.22,-1.91,745.0,NaN,746.0,-0.16,0.33,0.89,-0.93,1.05,0.94,2.11
0.0025,-197.84,45.19,-4.38,0.21,-6.86,746.0,NaN,746.0,-0.29,0.99,0.48,-5.09,0.75,0.36,2.09
0.005,-292.09,43.36,-6.74,0.20,-9.65,746.0,NaN,746.0,-0.30,0.98,0.32,-6.92,0.56,0.18,2.79
0.01,-265.98,39.63,-6.71,0.18,-8.89,713.0,NaN,746.0,-0.30,0.95,0.32,-6.82,0.46,0.15,1.65
0.05,-34.64,26.96,-1.28,0.12,-2.13,642.0,NaN,746.0,-0.16,0.89,0.81,-1.68,0.99,0.80,3.43
0.1,-38.79,39.00,-0.99,0.18,-2.29,726.0,NaN,746.0,-0.17,0.95,0.85,-1.36,0.92,0.78,1.40


In [54]:
build_table3(transaction_costs, cca_dict, ptf_ret)

,TO,GL,PL,IR_net,herf,pos
sample,10.45,12.25,0.51,1.05,0.43,49.32
0.0005,8.96,11.85,0.51,-1.11,0.46,49.25
0.001,7.79,11.52,0.51,-2.77,0.49,49.30
0.0025,5.39,10.72,0.51,-5.86,0.57,49.07
0.005,3.24,9.78,0.51,-7.63,0.68,48.70
0.01,1.42,8.34,0.51,-7.11,0.92,49.07
0.05,0.03,8.10,0.48,-1.30,0.90,51.95
0.1,0.00,11.66,0.53,-1.00,0.46,46.76


In [55]:
plotting(pnl_results, transaction_costs)